# BUSINESS UNDERSTANDING

In [ ]:
from IPython.display import Image
Image( 'img/CRISP.png' )

# 0 IMPORTS

In [ ]:
import pandas as pd
import inflection
import math
import numpy as np
import datetime
from scipy import stats as ss


from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('once')

from IPython.core.display import HTML
from IPython.display import Image


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.ensemble      import RandomForestRegressor
from sklearn.metrics       import mean_absolute_error, mean_squared_error
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.ensemble      import RandomForestRegressor
import xgboost as xgb


from boruta import BorutaPy

## 0.1 Helper Functions

In [ ]:
def cramer_v( x, y ):
    cm = pd.crosstab( x, y ).values
    n = cm.sum()
    r, k = cm.shape
    chi2 = ss.chi2_contingency( cm )[0]
    
    # Cramer V overstimation correction.
    chi2_ = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    k_ = k - (k-1)**2/(n-1)
    r_ = r - (r-1)**2/(n-1)
    
    
    return np.sqrt( (chi2_/n) / ( min( k_-1, r_-1 ) ) )



def jupyter_settings():
    %matplotlib inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [ 25, 12 ]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>' ) )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()



def mean_absolute_percentage_error( y, yhat ):
    return np.mean( np.abs( ( y - yhat ) / y ) )



def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
    return pd.DataFrame( { 'Model Name': model_name,
                           'MAE': mae,
                           'MAPE': mape,
                           'RMSE': rmse }, index=[0] )

In [ ]:
jupyter_settings()

## 0.2 Loading Data

CRISP-DS: DATA EXTRACTION (usually comes from SQL queries to here)

In [ ]:
df_sales_raw = pd.read_csv( 'data/train.csv', low_memory=False )
df_store_raw = pd.read_csv( 'data/store.csv', low_memory=False )

# merge
df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store')

In [ ]:
df_raw.sample()

# 1 DATA DESCRIPTION

CRISP-DS: DATA CLEANING

Describe data here. But first, copy it.

In [ ]:
df1 = df_raw.copy()

## 1.1 Rename Columns

Renaming columns can make life easier through the project. Check it out!

In [ ]:
df1.columns

In [ ]:
cols_old = df1.columns

snakecase = lambda x: inflection.underscore( x )

cols_new = list( map( snakecase, cols_old ) )

# rename
df1.columns = cols_new

In [ ]:
df1.columns

## 1.2 Data Dimensions

Here we define the proper tools to deal with the dataset. Is my computer enough? Do I need a cloud server or use Spark?

In [ ]:
print( 'Number of rows: {}'   .format( df1.shape[0] ) )
print( 'Number of columns: {}'.format( df1.shape[1] ) )

## 1.3 Data Types

Is your dataset assigned with proper data types? Usually Pandas assign non numbers as objects, so we fix it here if necessary.

Even though we cannot reassign type of columns that have NA values, it's good to preparate those that are ready.

In [ ]:
type_value_analysis = pd.concat([df1.dtypes, df1.isna().sum()], axis=1, keys=['Data Types', 'Missing Values'])
print(type_value_analysis)

In [ ]:
# Reassigning the column 'date' as datetime.
df1['date'] = pd.to_datetime( df1['date'] )

# Even though columns of week, month and year are assigned with float, we cannot change it due to missing values in it.
df1.dtypes

## 1.4 Check NA

What's the amount of NA?

What's the reason for them?

Depending on how it appears, there are some approaches we can take.

In [ ]:
df1.isna().sum()

## 1.5 Fillout NA

Some approaches are:
- Delete them: fast and easy! (but can backfire, then experiment other option too)
- Use ML slgorithms to fill it.
- Analyze the Business context, information and come up with solutions.

In [ ]:
# competition_distance: distance in meters to the nearest competitor store

# Hypothesis:
# 1. There is no competitor;
# 2. They are too far away.

# Solution #01:
# Change missing data to a distance waaay bigger than any other competitor distance.


# Take max distance:
max_dist = df1['competition_distance'].max()

# Make the NA values significantly bigger distance than max_dist (times 3 in this case).
df1['competition_distance'] =  df1['competition_distance'].apply( lambda x: (3*max_dist) if math.isnan( x ) else x )

In [ ]:
# Check NA again for competition_distance
df1.isna().sum()

In [ ]:
# competition_open_since_month/year: gives the approximate month/year of the time the nearest competitor was opened

# Hypothesis:
# 1. Competition opened store before us;
# 2. It was simply not record.

# Solution #01:
# Use the column 'date' as reference for the opening of competition and watch the impact on the ML algorithms.

df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month'] ) else x['competition_open_since_month'], axis=1 )
df1['competition_open_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan( x['competition_open_since_year'] ) else x['competition_open_since_year'], axis=1 )

In [ ]:
df1.isna().sum()

In [ ]:
# promo2_since_week/year: describes the year and calendar week when the store started participating in Promo2


# Hypothesis:
# 1. promo2 == 0.

# Check hypothesis:
# df1.loc[ df1['promo2'] == 0, ['promo2_since_week', 'promo2_since_year'] ].isna().sum()
# Looks perfect!

# Solution #01:
# Fill NA with 'date' values.

df1['promo2_since_week'] = df1.apply( lambda x: x['date'].week if math.isnan( x['promo2_since_week'] ) else x['promo2_since_week'], axis=1 )
df1['promo2_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan( x['promo2_since_year'] ) else x['promo2_since_year'], axis=1 )

In [ ]:
df1.isna().sum()

In [ ]:
# promo_interval: describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. 
# E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


# Hypothesis:
# 1. promo2 == 0

# Check hypothesis?
# df1.loc[ df1['promo2'] == 0, [ 'promo_interval'] ].isna().sum()
# Looks perfect!

# Solution #01:
# Fill NA with zeros: no promotion interval.

df1['promo_interval'].fillna( 0, inplace=True )

In [ ]:
# CHECKING IF CONTINUING PROMOTION ('promo2') IS HAPPENING IN EACH SALE

# promo_interval: describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. 
# E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

# Map month of each sale.
month_map = { 1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec' }
df1['month_map'] = df1['date'].dt.month.map( month_map )

# Define if there is a promo in each sale. Create a new column indicating if a sale is done with or without promotion.
df1['is_promo'] = df1[[ 'promo_interval', 'month_map' ]].apply( lambda x: ( 0 if x['promo_interval'] == 0 
                                                                            else ( 1 if x['month_map'] in x['promo_interval'].split( ',' ) 
                                                                                   else 0 ) ), axis=1 )

In [ ]:
# Check if NAs are gone.
df1.isna().sum()

## 1.6 Change types

After dealing with missing values, we can fix every column type properly.

In [ ]:
df1.dtypes

In [ ]:
# Reassign week, month and year columns as integers.
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype( int )
df1['competition_open_since_year']  = df1['competition_open_since_year'].astype( int )
df1['promo2_since_week']            = df1['promo2_since_week'].astype( int )
df1['promo2_since_year']            = df1['promo2_since_year'].astype( int )

In [ ]:
df1.dtypes

## 1.7 Descriptive Statistical Analysis

Métricas de: 
- Tendência central
- Dispersão

Encontrar erros grotescos

In [ ]:
num_attributes =  df1.select_dtypes( include=[ 'int32', 'int64', 'float64' ] )
cat_attributes =  df1.select_dtypes( exclude=[ 'int32', 'int64', 'float64', 'datetime64[ns]' ] )


In [ ]:
cat_attributes.sample(3)

### 1.7.1 Numerical Attributes

In [ ]:
# Central Tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

m = pd.concat( [ d2, d3, d4, ct1, ct2, d1, d5, d6 ] ).T.reset_index()
m.columns = [ 'attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis' ]
m

In [ ]:
sns.distplot( df1['sales'] )

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply( lambda x: x.unique().shape[0] )

In [ ]:
aux1 = df1[ ( df1['state_holiday'] != '0' ) & ( df1['sales'] != 0) ]


# Plotting boxplot
fig, axes = plt.subplots(1, 3, figsize=(12, 5), gridspec_kw={'wspace': 0.5})

plt.subplot( 1, 3, 1 )
sns.boxplot( x='state_holiday', y='sales', data=aux1 )

plt.subplot( 1, 3, 2 )
sns.boxplot( x='store_type', y='sales', data=aux1 )

plt.subplot( 1, 3, 3 )
sns.boxplot( x='assortment', y='sales', data=aux1 )

plt.show()

# 2 FEATURE ENGINEERING

CRISP-DS: DATA CLEANING

Making new Features here. But first, copy dataframe.

In [ ]:
df2 = df1.copy()

Why do it now?

## 2.0 Hypothesis Mindmap

MINDMAP DE HIPÓTESES
- qual fenômeno estou modelando? Daily Store Sales
- quem são os agentes atuantes sobre o fenômeno
- qual a descrição dos agentes?
- Validação ou não das hipóteses: surpresa ou quebra de crença

In [ ]:
# This should come from BUSINESS UNDERSTANDING and will help now
Image( 'img/mindmap_hipothesis.png' )

HIPÓTESES SÃO APOSTAS:

LOJAS MAIORES - MAIS VENDA

MAIOR VARIEDADE - MAIS VENDA

COMPETIDORES MAIS PERTO - MENOS VENDA

## 2.1 Hypothesis Creation

### 2.1.1 Store hypothesys

**1.** Mais funcionários - mais vendas

**2.** Mais estoque - mais venda

**3.** Maior porte - maior venda

**4.** Maior variedade - maior venda

**5.** Competidores mais próximos - menos venda

**6.** Competidores há mais tempo - mais venda

### 2.1.2 Product hypothesis

**1.** Mais marketing - mais venda

**2.** Mais exposição - mais venda

**3.** Preço menor - mais venda

**4.** Promoção maior / mais desconto - maior venda

**5.** Promoções mais longas - maior venda

**6.** Mais dias totais de promoção - mais venda

**7.** Promoções consecutivas - mais venda

### 2.1.3 Seasonality hypothesis

**1.** Lojas abertas no natal - mais venda

**2.** Loja vende mais ao longo dos anos

**3.** Mais venda no segundo semestre

**4.** Vender mais depois do dia 10 de cada mês

**5.** Vender menos no fim de semana

**6.** Vender menos nos feriados escolares

## 2.2 Final list of Hypothesis

PRODUTOS

**4.** Maior variedade - maior venda

**5.** Competidores mais próximos - menos venda

**6.** Competidores mais recentes - menos venda

LOJAS

**5.** Promoções recorrentes há mais anos - maior venda

**6.** Mais dias totais de promoção - mais venda

**7.** Promoções consecutivas - mais venda

SAZONALIDADE

**1.** Lojas abertas no natal - mais venda

**2.** Loja vende mais ao longo dos anos

**3.** Mais venda no segundo semestre (foco em dezembro)

**4.** Vender mais nos 10 primeiros dias de cada mês (salário recebido)

**5.** Vender menos no fim de semana

**6.** Vender menos nos feriados escolares

## 2.3 Feature Engineering

In [ ]:
# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week_of_year
df2['week_of_year'] = df2['date'].dt.isocalendar().week.astype( int )


# year_week
df2['year_week'] = df2['date'].dt.strftime( '%Y-%W' )



# competition_since
df2['competition_since'] = df2.apply( lambda x: datetime.datetime( year=x['competition_open_since_year'], month=x['competition_open_since_month'], day=1 ), axis=1 )
df2['competition_time_in_months'] = ( ( df2['date'] - df2['competition_since'] ) / 30 ).apply( lambda x: x.days ).astype( 'int64' )


# promo_since
df2['promo_since'] = df2['promo2_since_year'].astype( str ) + '-' + df2['promo2_since_week'].astype( str )
# "%Y-%W-%w has week 1 as the week after the year's first sunday.
df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime( x + '-1', "%Y-%W-%w" ) - datetime.timedelta( days=7 ) )
df2['promo_time_week'] = ( ( df2['date'] - df2['promo_since'] ) / 7 ).apply( lambda x: x.days ).astype( 'int64' )


# assortment
df2['assortment'] = df2['assortment'].apply( lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended' )

# state_holiday
df2['state_holiday'] = df2['state_holiday'].apply( lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day' )

In [ ]:
df2.dtypes

In [ ]:
df2.sample(5).T

# 3 FILTRAGEM DE VARIÁVEIS

CRISP-DS: DATA CLEANING

In [ ]:
df3 = df2.copy()

FILTRAGEM DE VARIÁVEIS

- Restrições do negócio

SELEÇÃO DE VARIÁVEIS 
- Variáveis mais relevantes para o MODELO

In [ ]:
df3.head().T

## 3.1 Filtragem das linhas

In [ ]:
df3 = df3[ ( df3['open'] != 0 ) & ( df3['sales'] > 0 ) ]

In [ ]:
# Exemplo de restrição: certa categoria tem 80% das vendas, beleza e cosméticos. Faz-se uma análise usando só esta categoria.

## 3.2 Seleção das colunas

In [ ]:
# Deletando 'open' porque só tem valor == 1,
# Deletando 'promo_interval' e 'month_map' porque já foram usada para criar 'is_promo',
# Deletando 'customers' porque não teremos informação de quantos clientes teremos.
# Assim como 'customers', checar outras colunas que não existem em test.csv
cols_drop = [ 'customers', 'open', 'promo_interval', 'month_map' ]
df3 = df3.drop( cols_drop, axis=1 )

In [ ]:
df3.columns

# 4 EXPLORATORY DATA ANALYSIS - EDA

CRISP-DS: EDA

In [ ]:
df4 = df3.copy()

OBJETIVOS:

1 - Ganhar experiência de negócio;

2 - Validar/Contrapor hipóteses de negócio (insights);

3 - Perceber variáveis que são importantes para o modelo.

## 4.1 Univariate Analysis

Como é a cara dessa variável? min, max, range, distribuição...

### 4.1.1 Response Variable

In [ ]:
sns.histplot( df4['sales'] )

In [ ]:
print( min( df4['sales'] ) )
print( max( df4['sales'] ) )
print( df4['sales'].mean() )
print( df4['sales'].median() )

### 4.1.2 Numerical Variable

In [ ]:
num_attributes.hist( bins=20 );

### 4.1.3 Categorical Variable

In [ ]:
df4['state_holiday'].unique()

In [ ]:
df4['store_type'].unique()

In [ ]:
df4['assortment'].unique()

In [ ]:
# Configuração da figura com espaçamento vertical ajustável
fig, axes = plt.subplots(3, 2, figsize=(12, 15), gridspec_kw={'height_ratios': [1, 1, 1]})

# Gráfico 1
a = df4[df4['state_holiday'] != 'regular_day']
sns.countplot(x='state_holiday', data=a, ax=axes[0, 0])

# Gráfico 2
sns.kdeplot(df4[df4['state_holiday'] == 'public_holiday']['sales'], label='public_holiday', fill=True, ax=axes[0, 1])
sns.kdeplot(df4[df4['state_holiday'] == 'easter_holiday']['sales'], label='easter_holiday', fill=True, ax=axes[0, 1])
sns.kdeplot(df4[df4['state_holiday'] == 'christmas']['sales'], label='christmas', fill=True, ax=axes[0, 1])

# Gráfico 3
sns.countplot(x='store_type', data=df4, ax=axes[1, 0])

# Gráfico 4
sns.kdeplot(df4[df4['store_type'] == 'a']['sales'], label='a', fill=True, ax=axes[1, 1])
sns.kdeplot(df4[df4['store_type'] == 'b']['sales'], label='b', fill=True, ax=axes[1, 1])
sns.kdeplot(df4[df4['store_type'] == 'c']['sales'], label='c', fill=True, ax=axes[1, 1])
sns.kdeplot(df4[df4['store_type'] == 'd']['sales'], label='d', fill=True, ax=axes[1, 1])

# Gráfico 5
sns.countplot(x='assortment', data=df4, ax=axes[2, 0])

# Gráfico 6
sns.kdeplot(df4[df4['assortment'] == 'basic']['sales'], label='basic', fill=True, ax=axes[2, 1])
sns.kdeplot(df4[df4['assortment'] == 'extended']['sales'], label='extended', fill=True, ax=axes[2, 1])
sns.kdeplot(df4[df4['assortment'] == 'extra']['sales'], label='extra', fill=True, ax=axes[2, 1])

# Ajuste do layout
plt.tight_layout()
plt.show()

## 4.2 Bivariada

Como certa variável impacta a variável resposta? Correlação, validação das hipóteses...

#### H1: Maior variedade - maior venda
**VERDADEIRA** Lojas 'extra' e 'extended' vendem mais que a 'basic', MAS tem bem menos lojas com maior variedade.

In [ ]:
plt.rcParams['figure.figsize'] = [13, 5]

aux1 = df4[ [ 'assortment', 'sales' ] ].groupby( 'assortment' ).mean().reset_index()
sns.barplot( x='assortment', y='sales', data=aux1 );

# assumindo que o 'extra' é o com maior variedade de produtos


aux2 = df4[ [ 'year_week', 'assortment', 'sales' ] ].groupby( [ 'year_week', 'assortment' ] ).mean().reset_index()
aux3 = aux2.pivot( index='year_week', columns='assortment', values='sales' )
aux3.plot()

#### H2: Competidores mais próximos - menos venda
**FALSA** Não há correlação aparente.

In [ ]:
plt.rcParams['figure.figsize'] = [13, 15]

aux1 = df4[ [ 'competition_distance', 'sales' ] ].groupby( 'competition_distance' ).mean().reset_index()

plt.subplot( 3, 1, 1 )
sns.scatterplot( x='competition_distance', y='sales', data=aux1 )

plt.subplot( 3, 1, 2 )
bins = list( np.arange( 0, 20000, 2000 ) )
aux2 = aux1
aux2[ 'competition_distance_binned' ] = pd.cut( aux2['competition_distance'], bins=bins )
aux2 = aux2[ [ 'competition_distance_binned', 'sales' ] ].groupby( 'competition_distance_binned' ).mean().reset_index()
sns.barplot( x='competition_distance_binned', y='sales', data=aux2 );
# plt.xticks( rotation=90 )

plt.subplot( 3, 1, 3 )
aux1 = df4[ [ 'competition_distance', 'sales' ] ].groupby( 'competition_distance' ).mean().reset_index()
sns.heatmap( aux1.corr( method='pearson' ), annot=True )

#### H3: Competidores há menos tempo - menos
**FALSA** Vende mais quanto mais recente a competição.

In [ ]:
plt.rcParams['figure.figsize'] = [13, 15]

plt.subplot( 3, 1, 1 )
aux1 = df4[ [ 'competition_time_in_months', 'sales' ] ].groupby('competition_time_in_months').sum().reset_index()
aux2 = aux1[ ( aux1['competition_time_in_months'] < 150 ) & ( aux1['competition_time_in_months'] > 0 ) ]
sns.barplot( x='competition_time_in_months', y='sales', data=aux2 );

plt.subplot( 3, 1, 2 )
sns.regplot( x='competition_time_in_months', y='sales', data=aux2 )

plt.subplot( 3, 1, 3 )
sns.heatmap( aux2.corr( method='pearson' ), annot=True )

### H4: Promoções recorrentes há mais anos - maior venda
**FALSA** Nos primeiros anos, temos um crescimento ínfimo nas vendas e depois elas caem drasticamente.

In [ ]:
plt.rcParams['figure.figsize'] = [13, 5]

# aux0 = df4[ df4['is_promo'] == 1 ]
aux0 = df4
aux1 = aux0[ [ 'promo_time_week', 'sales' ] ].groupby('promo_time_week').sum().reset_index()

bins = list( np.arange( -156, 520, 52 ) )
aux2 = aux1[ aux1['promo_time_week'] > 0 ]
aux2[ 'promo_time_week_binned' ] = pd.cut( aux2['promo_time_week'], bins=bins )
aux2 = aux2[ [ 'promo_time_week_binned', 'sales' ] ].groupby( 'promo_time_week_binned' ).mean().reset_index()
sns.barplot( x='promo_time_week_binned', y='sales', data=aux2 );

### <s>H5: Mais dias totais de promoção - mais venda</s>

In [ ]:
# ON HOLD


### H6: Promoções recorrentes - mais venda
**FALSA** Lojas com promo2 vendem menos.

In [ ]:
df4[ [ 'promo', 'promo2', 'sales' ] ].groupby( [ 'promo', 'promo2' ] ).mean().reset_index()

In [ ]:
aux1 = df4[ ( df4['promo'] == 1 ) & ( df4['promo2'] == 1 ) ][ [ 'year_week', 'sales' ] ].groupby('year_week').sum().reset_index()
ax = aux1.plot()

aux2 = df4[ ( df4['promo'] == 1 ) & ( df4['promo2'] == 0 ) ][ [ 'year_week', 'sales' ] ].groupby('year_week').sum().reset_index()
aux2.plot( ax=ax )

ax.legend( labels=['Tradicional e Extendida', 'Apenas tradicional'] );

### H7: Lojas abertas nos feriados - mais venda
**VERDADEIRA** Realmente vendem mais, aparentemente 10% a mais.

In [ ]:
aux1 = df4[ df4['state_holiday'] != 'regular_day' ]
aux1 = aux1[ [ 'state_holiday', 'sales' ] ].groupby( 'state_holiday' ).mean().reset_index()
sns.barplot( x='state_holiday', y='sales', data=aux1 );

### H8: Loja vende mais ao longo dos anos.
**FALSA** Houve uma queda anual. **MAS** houve um crescimento nas vendas nos primeiros semestres **E** uma queda grande no último trimestre.

**FOCO** para as vendas do último trimestre de 2015, que vem à frente.

In [ ]:
aux1 = df4[ [ 'year', 'sales' ] ].groupby('year').sum().reset_index()
sns.barplot( x='year', y='sales', data=aux1 );

In [ ]:
aux1 = df4[ df4['month'] <= 6 ]
aux1 = aux1[ [ 'year', 'sales' ] ].groupby('year').sum().reset_index()
sns.barplot( x='year', y='sales', data=aux1 );

In [ ]:
aux1 = df4[ df4['month'] > 7 ]
aux1 = aux1[ [ 'year', 'sales' ] ].groupby('year').sum().reset_index()
sns.barplot( x='year', y='sales', data=aux1 );

### H9: Mais venda no segundo semestre
**VERDADEIRA** Principalmente em Dezembro, as vendas são bem maiores.

In [ ]:
aux1 = df4.loc[ df4['year'] != 2015, [ 'month', 'sales' ] ].groupby('month').sum().reset_index()

plt.subplot( 1, 2, 1 )
sns.barplot( x='month', y='sales', data=aux1 )

plt.subplot( 1, 2, 2 )
sns.regplot( x='month', y='sales', data=aux1 )

### H10: Vende-se nos 10 primeiros dias: recebe-se salário.
**VERDADEIRA** Dividindo o mês em 3, o primeiro terço é o de maior venda.

In [ ]:
aux1 = df4[ [ 'day', 'sales' ] ].groupby('day').sum().reset_index()

plt.subplot( 1, 2, 1 )
sns.barplot( x='day', y='sales', data=aux1 )


plt.subplot( 1, 2, 2 )
bins = list( np.arange( 0, 31, 10 ) )
aux2 = aux1
aux2[ 'days_binned' ] = pd.cut( aux2['day'], bins=bins )
aux2 = aux2[ [ 'days_binned', 'sales' ] ].groupby( 'days_binned' ).mean().reset_index()
sns.barplot( x='days_binned', y='sales', data=aux2 );

### H11: Vender menos no fim de semana.
**VERDADEIRA** Vendem menos nos dias 6 e 7.

In [ ]:
aux1 = df4[ [ 'day_of_week', 'sales' ] ].groupby('day_of_week').sum().reset_index()

plt.subplot( 1, 2, 1 )
sns.barplot( x='day_of_week', y='sales', data=aux1 )

plt.subplot( 1, 2, 2 )
sns.regplot( x='day_of_week', y='sales', data=aux1 )

### H12: Vender menos nos feriados escolares.
**FALSA** Comparando a média de venda em dias com e sem aula, os sem aula levam vantagem.

In [ ]:
aux1 = df4[ [ 'school_holiday', 'sales' ] ].groupby('school_holiday').mean().reset_index()

plt.subplot( 1, 2, 1 )
sns.barplot( x='school_holiday', y='sales', data=aux1 )

### 4.2.1 Hipothesis Summary

In [ ]:
tab = [ [ 'Hypothesis', 'Conclusion', 'Relevance' ],
        [ 'H1 - assortment', 'True', 'Low' ],
        [ 'H2 - competitors_distance', 'False', 'Medium' ],
        [ 'H3 - competitors_since', 'False', 'Medium' ],
        [ 'H4 - promo_time_week', 'False', 'Low' ],
        [ 'H5 - ', '-', '-' ],
        [ 'H6 - promo2', 'False', 'High' ],
        [ 'H7 - state_holiday', 'True', 'Low' ],
        [ 'H8 - year', 'False', 'High' ],
        [ 'H9 - month', 'True', 'High' ],
        [ 'H10 - day', 'True', 'Medium' ],
        [ 'H11 - day_of_week', 'True', 'High' ],
        [ 'H12 - school_holiday', 'False', 'Low' ],
      ]

hypothesis_summary = tabulate( tab, headers='firstrow' )
print( hypothesis_summary )

## 4.3 Multivariada

Como as variáveis se relacionam 2:1, 3:1, etc com a variável resposta?

### 4.3.1 Numerical attributes

In [ ]:
df4.columns

In [ ]:
num_attributes = num_attributes.drop([ 'open', 'customers' ], axis=1)

In [ ]:
num_attributes.columns

In [ ]:
correlation = num_attributes.corr(method='pearson')
sns.heatmap(correlation, annot=True, fmt=".2f")

### 4.3.2 Categorical attributes

In [ ]:
cat_attributes = df4.select_dtypes( include='object' )
cat_attributes = cat_attributes.drop( 'year_week', axis=1 )

In [ ]:
cat_attributes.head()

In [ ]:
# Selecting categorical data
cat_attributes = df4.select_dtypes( include='object' )
cat_attributes = cat_attributes.drop( 'year_week', axis=1 )

# Calculating Cramer V matrix.
a1 = cramer_v( cat_attributes['state_holiday'], cat_attributes['state_holiday'] )
a2 = cramer_v( cat_attributes['state_holiday'], cat_attributes['store_type'] )
a3 = cramer_v( cat_attributes['state_holiday'], cat_attributes['assortment'] )

a4 = cramer_v( cat_attributes['store_type'], cat_attributes['state_holiday'] )
a5 = cramer_v( cat_attributes['store_type'], cat_attributes['store_type'] )
a6 = cramer_v( cat_attributes['store_type'], cat_attributes['assortment'] )

a7 = cramer_v( cat_attributes['assortment'], cat_attributes['state_holiday'] )
a8 = cramer_v( cat_attributes['assortment'], cat_attributes['store_type'] )
a9 = cramer_v( cat_attributes['assortment'], cat_attributes['assortment'] )

# Cramer V matrix
d = pd.DataFrame( { 'state_holiday': [a1, a2, a3],
                    'store_type':    [a4, a5, a6], 
                    'assortment':    [a7, a8, a9] } )

d = d.set_index( d.columns )

sns.heatmap( d, annot=True, cmap="YlGnBu" )

# 5 DATA PREPARATION

CRISP-DS: DATA MODELING

## 5.0 Project status and Topic meaning

In [ ]:
Image( 'img/progresso.png' )

In [ ]:
df5 = df4.copy()

Os modelos de ML funcionam melhor com dados numéricos e na mesma escala.

- **1 -** Normalization (para Gaussiana): Rescala o centro para zero com desvio padrão igual a 1.
- **2 -** Rescaling (para não Gaussiana): Rescala para um intervalo de 0 a 1.
- **3 -** Transformation: Encoding (categorical to numerical), and data nature transformation (months are cyclical, for exemple).

## 5.1 Standardization

x_new = (x_i - media)/desv

Analyzing topic 4.1 Univariate Analysis, we cannot see clearly any variate available for normalization.

In [ ]:
# from sklearn.preprocessing import StandardScaler

# # Assuming 'data' is your dataset
# scaler = StandardScaler()
# data_standardized = scaler.fit_transform(data)

## 5.2 Rescaling

In [ ]:
num_feat = df5.select_dtypes( include=[ 'int32', 'int64', 'float64' ] )
num_feat.columns

Robust Scaler (data with strong outliers) or Min-Max Scaler (no outliers)

In [ ]:
# Testing Outliers
#sns.boxplot(x=df5['promo_time_week'])
#plt.xlabel("x")
#plt.show();

In [ ]:
# LINEAR VARIABLES
mms = MinMaxScaler()
rs = RobustScaler()

# year
df5['year'] = mms.fit_transform( df5[['year']].values )

# competition_distance - Robust
df5['competition_distance'] = rs.fit_transform( df5[['competition_distance']].values )

# competition_time_in_months
df5['competition_time_in_months'] = rs.fit_transform( df5[['competition_time_in_months']].values )

# promo_time_week
df5['promo_time_week'] = mms.fit_transform( df5[['promo_time_week']].values )

## 5.3 Encoding

### 5.3.1 Categorical Encoding

In [ ]:
# One Hot Encoding (google image it to check)
# category  - >  hot    warm    cold
#   hot           1      0       0
#  cold           0      0       1
#  warm           0      1       0


# Label Encoding (just change names to numbers)
# red, yellow, blue -> 1, 2, 3


# Ordinal Encoding (change names to numbers respecting an order)
# warm, cold, hot -> 2, 1, 3


# Target Encoding (applied in features with many categories)
# Its name is a hint: it works with the target variable (in our case, it's sales).
# Stores a, b, c, d, e, f, g, h, i, f, etc...
# Total Sales: 100
# 'a' sales: 12. New 'a' value: 0.12
# 'b' sales: 2. New 'b' value: 0.02
# 'c' sales: 4. New 'c' value: 0.04
# ...
# 'f' sales: 9. New 'f' value: 0.09
# 'g' sales: 12. New 'g' value: 0.12
# ...


# Frequency Encoding (applied in features with many categories)
# It takes the category appearence frequency as its new value.



# Embedding Encoding (used in NLP)
# Analyzes the context of which the categories appear.



In [ ]:
# state_holiday - One Hot Encoding
df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'], dtype=int)

# store_type - Label Encoding
le = LabelEncoder()
df5['store_type'] = le.fit_transform( df5['store_type'] )


# assortment - Ordinal Encoding
assort_dict = { 'basic': 1, 'extra': 2, 'extended': 3 }
df5['assortment'] = df5['assortment'].map( assort_dict )

### 5.3.2 Response Variable Transformation

Transforming Poisson into Gaussian

- Logarithm
- Box-Cox
- Cube-root
- Sqrt
- Sine/Cosine

In [ ]:
df5['sales'] = np.log1p( df5['sales'] )

In [ ]:
sns.distplot( df5['sales'] );

### 5.3.3 Cyclical Features Encoding

In [ ]:
# CYCLICAL FEATURES ENCODING

# day_of_week
df5['dayweek_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2. * np.pi/7 ) ) )
df5['dayweek_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2. * np.pi/7 ) ) )

# month
df5['month_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2. * np.pi/12 ) ) )
df5['month_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2. * np.pi/12 ) ) )

# day
df5['daymonth_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2. * np.pi/30 ) ) )
df5['daymonth_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2. * np.pi/30 ) ) )

# week_of_year
df5['weekyear_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
df5['weekyear_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )

# 6 FEATURE SELECTION

## 6.0 

CRISP-DS: DATA MODELING

In [ ]:
df6 = df5.copy()

Occam's Razor: O mais simples é essencial para um algoritmo de ML.

In [ ]:
# FILTER METHODS (Univariada)

#Relevância através da correlação entre variáveis

#             |       Numérica       |                 Categórica
#----------------------------------------------------------------------------------
# Numérica    |    corr. Pearson     |     LDA (linear discriminant analysis)
#----------------------------------------------------------------------------------
# Categórica  |        Anova         |           Chi-Square / Cramer's V

# Não considera a força de múltiplas variáveis que se reforçam para agir na variável alvo.

In [ ]:
# EMBEDDED METHODS (por importância / embutido)
# Já embutido em alguns algoritmos

# Random Forest
# Gini impurity: Determina o grau de importância de uma feature ao usá-la para dividir um conjunto de dados em dois subconjuntos. Se esses
# subconjuntos forem bastante homogêneos entre si, a feature tem bastante importância. Caso contrário, tem baixa importância.

# Lasso / Ridge -> Coefficients Importance

In [ ]:
# Wrapper Methods (por subset)

# Testa variável por variável em vários ciclos rodando algoritmos de ML. Se ao adicionar uma variável, a performance aumenta, ela é mantida. Caso 
# contrário, ela é removida.

# ALGORITMO BORUTA

## 6.1 Splitting dataframe between training and test dataset

In [ ]:
df6.head(1)

In [ ]:
cols_drop = [ 'week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week' ]
df6 = df6.drop( cols_drop, axis=1 )

In [ ]:
# Splitting last 6 weeks of sales as sample for testing.
six_week_ago = df6[['store', 'date']].groupby('store').max().reset_index()['date'][0] - datetime.timedelta( days = 6*7 )

In [ ]:
# training dataset
x_train = df6[ df6['date'] < six_week_ago ]
y_train = x_train['sales']

# test dataset
x_test = df6[ df6['date'] >= six_week_ago ]
y_test = x_test['sales']

print( 'Training Min Date: {}'.format( x_train['date'].min() ) )
print( 'Training Max Date: {}'.format( x_train['date'].max() ) )

print( '\nTest Min Date: {}'.format( x_test['date'].min() ) )
print( 'Test Max Date: {}'.format( x_test['date'].max() ) )

## 6.2 Boruta as Feature Selector

In [ ]:
x_train.head(1)

In [ ]:
# training and test dataset
x_train_n = x_train.drop( [ 'date', 'sales' ], axis=1 ).values
y_train_n = y_train.values.ravel()

# define RandomForestRegressor
rf = RandomForestRegressor( n_jobs=-1 )

# define Boruta
boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( x_train_n, y_train_n )

### 6.2.1 Best features from Boruta

In [ ]:
cols_selected = boruta.support_.tolist()

x_train_fs = x_train.drop( [ 'date', 'sales' ], axis=1 )
cols_selected_boruta = x_train_fs.iloc[ :, cols_selected ].columns

cols_not_selected_boruta = np.setdiff1d( x_train_fs.columns, cols_selected_boruta )

In [ ]:
list(cols_selected_boruta)

In [ ]:
cols_not_selected_boruta

In [ ]:
print(hypothesis_summary)

In [ ]:
# Compare hypothesis_summary and cols_selected_boruta

## 6.3 Manual Feature Selection

In [ ]:
# boruta features
cols_selected_boruta = [ 'store',
                         'promo',
                         'store_type',
                         'assortment',
                         'competition_distance',
                         'competition_open_since_month',
                         'competition_open_since_year',
                         'promo2',
                         'promo2_since_week',
                         'promo2_since_year',
                         'competition_time_in_months',
                         'promo_time_week']

# cols to add
feat_to_add = [ 'date', 'sales' ]

# final features
#cols_selected_boruta.extend( feat_to_add )

In [ ]:
cols_selected_boruta

# 7 MACHINE LEARNING ALGORITHMS

### Algoritmos supervisionados

TREINAMENTO: Compara outras entradas com variável alvo. Tenta identificar padrão.


TAREFA SUPERVISIONADA - VARIÁVEL ALVO ROTULADA e é usada no treinamento

1 - Classificação - analisa características disponíveis e divide os dados em categorias. Quando um novo dado é adicionado, coloca ele em uma das classificações.

2 - Regressão - analisa características disponíveis e divide os dados em valores contínuos. Quando um novo dado é adicionado, um novo valor é dado
a ele.

3 - Séries Temporais - aplicando Regressão a uma situação em que se tem informação de tempo corrido, é possível prever tendências futuras.

### Algoritmos não-supervisionados

Sem nenhum rótulo conhecido pelo algoritmo, ele tenta agrupar os dados.

Clusterização/Agrupamento

### Algoritmos semi-supervisionados

REINFORCEMENT LEARNING

Agente age sobre Ambiente
Ambiente recompensa Agente

Damn it, Pavlov.

Exemplo: algoritmo do Netflix

## ML Models

LINEARES

**1 -** Average Model

**2 -** Linear Regression

**3 -** Linear Regression Regularized

NÃO-LINEARES

**4 -** Random Forest Regression

**5 -** XGBoost Regressor

## 7.0 Data

In [ ]:
x_train_ml = x_train[ cols_selected_boruta ]
x_test_ml = x_test[ cols_selected_boruta ]

In [ ]:
x_train.shape

## 7.1 Average Model

In [ ]:
aux1 = x_test_ml.copy()
aux1['sales'] = y_test.copy()

# prediction
aux2 = aux1[ [ 'store', 'sales' ] ].groupby('store').mean().reset_index().rename( columns={'sales': 'predictions'} )
aux1 = pd.merge( aux1, aux2, how='left', on='store' )
yhat_baseline = aux1['predictions']


# performance
baseline_result = ml_error( 'Average Model', np.expm1(y_test), np.expm1(yhat_baseline) )
baseline_result

## 7.2 Linear Regression

In [ ]:
# Model
lr = LinearRegression().fit( x_train_ml, y_train )

# Prediction
yhat_lr = lr.predict( x_test_ml )

# Performance
lr_result = ml_error( 'Linear Regression', np.expm1(y_test), np.expm1(yhat_lr) )
lr_result

## 7.3 Linear Regression Regularized - Lasso

In [ ]:
# Model
lrr = Lasso( alpha=0.01 ).fit( x_train_ml, y_train )

# Prediction
yhat_lrr = lr.predict( x_test_ml )

# Performance
lrr_result = ml_error( 'Linear Regression - Lasso', np.expm1(y_test), np.expm1(yhat_lrr) )
lrr_result

## 7.4 Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor


# Model
rf = RandomForestRegressor( n_estimators=50, n_jobs=-1, random_state=42 ).fit( x_train_ml, y_train )

# Prediction
yhat_rf = rf.predict( x_test_ml )

# Performance
rf_result = ml_error( 'Random Forest Regressor', np.expm1(y_test), np.expm1(yhat_rf) )
rf_result

## 7.5 XGBoost Regressor

In [ ]:
import xgboost as xgb

# Model
model_xgb = xgb.XGBRegressor( objective='reg:squarederror',
                             n_estimators=1000, 
                             eta=0.01, 
                             max_depth=10,
                             subsample=0.7,
                             colsample_bytee=0.9 ).fit( x_train_ml, y_train )

# Prediction
yhat_xgb = model_xgb.predict( x_test_ml )

# Performance
xgb_result = ml_error( 'XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgb) )
xgb_result

## 7.6 Compare Models' Performances

In [ ]:
modeling_result = pd.concat( [ baseline_result, lr_result, lrr_result, rf_result, xgb_result ] )
modeling_result.sort_values( 'RMSE' )

## 7.7 Cross-Validation

# 0.0 IMPORTS

# 0.0 IMPORTS

# 0.0 IMPORTS